# Quark field renormalization

My usual workflow for writing code is to store utility functions in a ```.py``` script in the same folder, then import these functions into a Jupyter notebook. I've left some scaffolding code in ```utils.py``` that you should try to implement-- let me know if anything isn't clear or if you have any questions!

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
# from utils import *
import sys
sys.path.append('/Users/theoares/lqcd/utilities')
from pytools import *
from formattools import *
import plottools as pt

: 

## Read input data

Reads input data from an hdf5 file using the ```h5py``` module. Here ```cfgs``` will store the paths to all the configurations. You can read a file with ```f = h5py.File(file_name, 'r')```, where ```'r'``` is the flag to read the file. If you want to write an hdf5 file, use the ```'w'``` flag.

The name of the folder specifies the lattice parameters. ```cl``` means the clover action is used (ch 9 of Gattringer & Lang), 3 means there are three degenerate quark flavors (up, down, and strange), ```24_24``` is the lattice geometry of $24^3\times 24$ sites, ```b6p1``` means the gauge coupling is $\beta = 6.1$, and ```m0p2450``` means the three quarks have a degenerate mass of $am = -0.2450$. 

Here's the documentation for ```h5py``` if you're interested!

https://docs.h5py.org/en/stable/

In [2]:
# change in_dir to wherever on your computer the data is stored
in_dir = '/Users/theoares/Dropbox (MIT)/research/npr_momfrac/meas/cl3_24_24_b6p1_m0p2450_25291'
L, T = 24, 24        # lattice dimensions
LL = np.array([L, L, L, T])

cfgs = []
for (dirpath, dirnames, file) in os.walk(in_dir):
    cfgs.extend(file)
for idx, cfg in enumerate(cfgs):
    cfgs[idx] = in_dir + '/' + cfgs[idx]
n_cfgs = len(cfgs)
print('Number of configurations: \n' + str(n_cfgs))

Number of configurations: 
12


You can think of h5py files as containing datasets, where you can access each dataset like a dictionary with the correct key. For example, here's what one of the files in ```cfgs``` contains. We'll focus on the ```'prop'``` group, which contains propagators. The propagators are computed at a given momentum $p$,
$$
    S(p) = \sum_{x} e^{ip\cdot (x - y)} \langle q(x) \overline q(y) \rangle
$$
where $p^\mu$ is the 4-momentum specified in the key's tag. For example, the tag 'props/p2142' means this propagator is computed at momentum $k = (1, 1, 1, 1)$. Renormalization coefficients are computed at a given momentum $\mathcal Z(p)$, and later we can talk about why we want to know $\mathcal Z(p)$ at a large amount of momenta. 

Let's use the momentum mode $k = (1, 1, 1, 1)$ for this calculation. Later we'll generalize whatever code we have to more momenta! Note that $k$ is a wavevector and indexes a momentum mode, and the actual value of the momentum is:
$$
    p_\mu = \frac{2\pi k_\mu}{L_\mu}
$$
where $L_\mu = (L, L, L, T)$ is the number of sites in each direction on the lattice. We'll also use the lattice momentum $\tilde p_\mu$, defined as
$$
    \tilde{p}_\mu = \frac{2}{a} \sin(\frac{ap}{2})
$$
which is what you get for the momentum when you quantize it in finite volume. For small $ap$, the linear and lattice momentums are nearly identical because of $\sin x\sim x$. 

The propagator is a matrix in spinor space, and a matrix in color space, since each quark field has a spinor and a color index. It's therefore stored as a $(4, 4, 3, 3)$-dimensional tensor, and here we can see what it actually looks like numerically. We'll actually reshape it into a $(3, 4, 3, 4)$-dimensional tensor using the ```np.einsum``` function, as tensor-wise operations in the ```np.linalg``` library assume a square shape. The ```np.einsum``` function is your best friend for tensor manipulations, it essentially lets you do index notation on numpy arrays. Here's the documentation, I'd encourage you to check it out and play with a few examples: https://numpy.org/doc/stable/reference/generated/numpy.einsum.html

In [3]:
k = np.array([1, 1, 1, 1], dtype = np.float64)
p = 2*np.pi*k / LL
props = np.zeros((n_cfgs, 3, 4, 3, 4), dtype = np.complex64)
for ii, cfg in enumerate(cfgs):
    f = h5py.File(cfg, 'r')                             # read files
    prop_tmp = f['prop/p1111'][()]                      # [()] opens the dataset
    props[ii] = np.einsum('ijab->aibj', prop_tmp)        # store and reshape tensor
    f.close()                                           # close file
print(props.shape)

(12, 3, 4, 3, 4)


### Bootstrap input data

The propagator object $\{S_i(p)_{\alpha\beta}^{ab}\}_{i = 1}^{n_\mathrm{cfgs}}$ is stored as a $(n_\mathrm{cfgs}, 3, 4, 3, 4)$ dimensional tensor, where $i$ runs over configurations, $a, b$ are color indices, and $\alpha, \beta$ are spinor indices. To estimate the population distribution, we have to bootstrap the input data. The procedure of boostrapping takes the input data, computed on each Monte Carlo configuration, and returns a new distribution $\{S_b(p)_{\alpha\beta}^{ab}\}_{b = 1}^{n_\mathrm{boot}}$, where $n_\mathrm{boot}$ is usually picked to be close to $n_\mathrm{cfgs}$. We'll use $n_\mathrm{boot} = 20$.

To generate a bootstrap sample $S_b(p)$, pick $n_\mathrm{cfgs}$ samples $S_i(p)$ at random **with replacement**, $\{S_{i_1}(p), ..., S_{i_{n_\mathrm{cfgs}}}(p))\}$, where some of the $i_j$'s may be equal. You can do this with the ```np.random.choice``` function (https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html). Once this is done, the bootstrap sample is generated by averaging over these propagators
$$
    S_b(p) = \sum_{j = 1}^{n_\mathrm{cfgs}} S_{i_j}(p)
$$
Repeat this procedure $n_\mathrm{boot}$ times to get the bootstrap distribution.

In [4]:
n_boot = 20
prop_boot = bootstrap(props, n_boot)            # bootstrap() needs to be implemented in utils.py!

NotImplementedError: bootstrap needs to be implemented

### Compute $\mathcal Z(q)$

Once the propagators are bootstrapped, we need to compute the quark-field renormalization. This is done with the following definition:
$$
    \mathcal Z_q(p^2) = \frac{i}{12\tilde{p}^2} \mathrm{Tr} \left[ S^{-1}(p) \tilde p_\mu \gamma^\mu]) \right]
$$
The trace here is over spinor and color indices, and the easiest way to implement both the trace and the tensor multiplication is to use ```np.einsum```. I've given a basis for the $\gamma^\mu$ matrices that you can use in ```utils.py```, they're just the ```gamma``` variable.

In [ ]:
Zq = quark_renorm(prop_boot, p, n_boot)